In [23]:
import requests
from bs4 import BeautifulSoup

def validate_sparql_syntax(query: str) -> tuple[bool, str | None]:
    """
    Validates the syntax of a SPARQL query using the SPARQLer Query Validator.
    
    Parameters:
      query (str): The SPARQL query string to validate.
    
    Returns:
      tuple: (True, None) if the query is syntactically correct,
             (False, error_message) if there is a syntax error.
    """
    base_url = "http://www.sparql.org/"
    endpoint = base_url + "$/validate/query"
    
    data = {
        "query": query,
        "languageSyntax": "SPARQL",
        "outputFormat": "sparql",
        "linenumbers": "true"
    }
    
    headers = {
        "Referer": "http://www.sparql.org/query-validator.html",
    }
    
    try:
        response = requests.post(endpoint, data=data, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        return False, f"Request failed: {e}"
    
    # Parse the HTML response
    soup = BeautifulSoup(response.text, 'html.parser')

    #print(soup)
    
    # Check if there's a syntax error section
    error_section = soup.find('p', text='Syntax error:')
    
    if error_section:
        # Extract the error message
        error_box = error_section.find_next('pre', class_='box')
        if error_box:
            return False, error_box.text.strip()
        else:
            return False, "Unknown syntax error"
    else:
        # No error section means the query is valid
        return True, None

In [24]:
# Define a valid query
valid_query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name WHERE {
  ?person a foaf:Person .
  ?person foaf:name ?name .
}
"""

# Define an invalid query (intentionally missing a period to induce syntax error)
invalid_query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?name WHERE {
  ?person a foaf:Person 
  ?person foaf:name ?name .   # Missing period between triple patterns?
}
"""

# Test the valid query
is_valid, error = validate_sparql_syntax(valid_query)
if is_valid:
    print("✅ Valid query.")
else:
    print("❌ Invalid query:", error)

# Test the invalid query
is_valid, error = validate_sparql_syntax(invalid_query)
if is_valid:
    print("✅ Valid query.")
else:
    print("❌ Invalid query:", error)


/var/folders/cf/q7bl9m0529ncggwfh1ml6k5m0000gn/T/ipykernel_49555/1194369607.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  error_section = soup.find('p', text='Syntax error:')


✅ Valid query.
❌ Invalid query: Encountered "  "?person "" at line 5, column 3.
Was expecting one of:
    "values" ...
    "graph" ...
    "optional" ...
    "minus" ...
    "bind" ...
    "service" ...
    "filter" ...
    "{" ...
    "}" ...
    ";" ...
    "," ...
    "." ...
    "{|" ...
